This notebook mainly contain the RNNs parts left out in the main notebook, simply copy paste the part into the RNN part of main notebook to be able to run it smoothly.

## Table of Contents

    1. GRU with summary only
    2. LSTM with summary only
    3. LSTM

# 1. GRU with summary only

In [15]:
input_summary = Input(shape = (summary_pad_len, ))
summary_embeddings = Embedding(summary_vocab_size, output_dim=50,
                     embeddings_initializer = Constant(summary_glove_weights),
                     input_length = summary_pad_len, trainable = True)(input_summary)
GRU_summary = GRU(100)(summary_embeddings)
dense_summary = Dense(1, activation ="linear")(GRU_summary)

input_nontext = Input(shape=(X_train.shape[1],))


# Concatenate
concat = concatenate([dense_summary, input_nontext])
dense_full1 = Dense(256, activation = "relu")(concat)
dense_full2 = Dense(128, activation="relu")(dense_full1)
dense_full3 = Dense(64, activation="relu")(dense_full2)
output_layer = Dense(1, activation = "linear")(dense_full3)

model = Model(inputs=[input_summary, input_nontext], outputs = output_layer)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 407)]        0           []                               
                                                                                                  
 embedding (Embedding)          (None, 407, 50)      736850      ['input_1[0][0]']                
                                                                                                  
 gru (GRU)                      (None, 100)          45600       ['embedding[0][0]']              
                                                                                                  
 dense (Dense)                  (None, 1)            101         ['gru[0][0]']                    
                                                                                              

In [16]:
opt = keras.optimizers.Adam(learning_rate=0.01) 

model.compile(optimizer = opt, loss='mean_squared_error', metrics=['mse', 'mae'])

start = time.time()

batch_size = 128
epochs = 25

callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=0.01, restore_best_weights=True)]

story = model.fit([padded_X_train_summary, X_train], y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_split = 0.25,
                  callbacks = callbacks) 

end = time.time()

print(end - start)

Epoch 1/25
260/260 [==============================] - 546s 2s/step - loss: 3383.4844 - mse: 3383.4844 - mae: 35.9039 - val_loss: 2777.6506 - val_mse: 2777.6506 - val_mae: 31.4482
Epoch 2/25
260/260 [==============================] - 529s 2s/step - loss: 2787.5581 - mse: 2787.5581 - mae: 32.6156 - val_loss: 2665.2039 - val_mse: 2665.2039 - val_mae: 31.7357
Epoch 3/25
260/260 [==============================] - 533s 2s/step - loss: 2425.9875 - mse: 2425.9875 - mae: 30.4488 - val_loss: 2870.3379 - val_mse: 2870.3379 - val_mae: 31.3374
Epoch 4/25
260/260 [==============================] - 503s 2s/step - loss: 2139.4868 - mse: 2139.4868 - mae: 28.5085 - val_loss: 2765.9397 - val_mse: 2765.9397 - val_mae: 30.6319
Epoch 5/25
260/260 [==============================] - 185s 713ms/step - loss: 1993.0309 - mse: 1993.0309 - mae: 27.5109 - val_loss: 2696.0759 - val_mse: 2696.0759 - val_mae: 31.1165
Epoch 5: early stopping
2297.6020896434784


In [17]:
gru_sum_val_prediction = model.predict([padded_X_test_summary, X_test])
gru_sum_val_prediction = pd.DataFrame(gru_sum_val_prediction, columns = ["price"])
gru_sum_val_result = pd.concat([X_test_listing_id, gru_sum_val_prediction], axis = 1)

346/346 [==============================] - 30s 85ms/step


# 2. LSTM with summary only

In [ ]:
input_summary = Input(shape=(summary_pad_len, ))
summary_embeddings = Embedding(summary_vocab_size, output_dim=50,
                     embeddings_initializer=Constant(summary_glove_weights),
                     input_length=summary_pad_len, trainable=False)(input_summary)
LSTM_text1 = LSTM(100, return_sequences=False)(summary_embeddings)
dense_text = Dense(1, activation="linear")(LSTM_text1)


input_nontext = Input(shape=(X_train.shape[1],))

# Concatenate
concat = concatenate([dense_text, input_nontext])
dense_full1 = Dense(256, activation = "relu")(concat)
dense_full2 = Dense(128, activation="relu")(dense_full1)
output_layer = Dense(1, activation = "linear")(dense_full2)

model = Model(inputs=[input_summary, input_nontext], outputs = output_layer)
model.summary()

In [144]:
opt = keras.optimizers.Adam(learning_rate=0.01) 

model.compile(optimizer = opt, loss='mean_squared_error', metrics=['mse', 'mae'])

batch_size = 128
epochs = 25

callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=0.01, restore_best_weights=True)]

story = model.fit([padded_X_train_summary, X_train], y_train,
                    batch_size = batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split = 0.25,
               callbacks=callbacks) 

Epoch 1/25
260/260 [==============================] - 264s 1s/step - loss: 2752.2649 - mse: 2752.2649 - mae: 32.4861 - val_loss: 2830.1797 - val_mse: 2830.1797 - val_mae: 33.0500
Epoch 2/25
260/260 [==============================] - 254s 977ms/step - loss: 2729.9072 - mse: 2729.9072 - mae: 32.4006 - val_loss: 2739.5403 - val_mse: 2739.5403 - val_mae: 31.5569
Epoch 3/25
260/260 [==============================] - 254s 977ms/step - loss: 2682.1990 - mse: 2682.1990 - mae: 32.0406 - val_loss: 2769.9661 - val_mse: 2769.9661 - val_mae: 33.9945
Epoch 4/25
260/260 [==============================] - 267s 1s/step - loss: 2668.2625 - mse: 2668.2625 - mae: 32.0252 - val_loss: 2823.0339 - val_mse: 2823.0339 - val_mae: 35.5169
Epoch 5/25
260/260 [==============================] - 273s 1s/step - loss: 2685.4473 - mse: 2685.4473 - mae: 32.1307 - val_loss: 2715.7258 - val_mse: 2715.7258 - val_mae: 33.0590
Epoch 6/25
260/260 [==============================] - 265s 1s/step - loss: 2654.8853 - mse: 2654.88

In [151]:
lstm_sum_val_prediction = model.predict([padded_X_test_summary, X_test])
listing_id = pd.DataFrame(X_test_listing_id, columns = ["listing_id"])
lstm_sum_val_prediction = pd.DataFrame(lstm_sum_val_prediction, columns = ["price"])
lstm_sum_val_prediction.describe()

346/346 [==============================] - 44s 126ms/step


,price
count,11057.000000
mean,102.261246
std,63.995358
min,8.145638
25%,50.614307
50%,89.056229
75%,132.858215
max,454.106262


# 3. LSTM

In [ ]:
input_name = Input(shape=(name_pad_len, ))
name_embeddings = Embedding(name_vocab_size, output_dim=50,
                     embeddings_initializer = Constant(name_glove_weights),
                     input_length = name_pad_len, trainable = True)(input_name)   # try trainable False
LSTM_name1 = LSTM(25, return_sequences = False)(name_embeddings)
dense_name = Dense(1, activation ="linear")(LSTM_name1)


input_summary = Input(shape = (summary_pad_len, ))
summary_embeddings = Embedding(summary_vocab_size, output_dim=50,
                     embeddings_initializer = Constant(summary_glove_weights),
                     input_length = summary_pad_len, trainable = True)(input_summary)
LSTM_summary1 = LSTM(100, return_sequences = False)(summary_embeddings)
dense_summary = Dense(1, activation ="linear")(LSTM_summary1)


input_description = Input(shape = (description_pad_len, ))
description_embeddings = Embedding(description_vocab_size, output_dim=50,
                     embeddings_initializer = Constant(description_glove_weights),
                     input_length = description_pad_len, trainable = True)(input_description)
LSTM_description1 = LSTM(100, return_sequences = False)(description_embeddings)
dense_description = Dense(1, activation="linear")(LSTM_description1)


input_amenities = Input(shape = (amenities_pad_len, ))
amenities_embeddings = Embedding(amenities_vocab_size, output_dim=50,
                     embeddings_initializer = Constant(amenities_glove_weights),
                     input_length = description_pad_len, trainable = True)(input_amenities)
LSTM_amenities1 = LSTM(100, return_sequences = False)(amenities_embeddings)
dense_amenities = Dense(1, activation="linear")(LSTM_amenities1)


input_nontext = Input(shape=(X_train.shape[1],))


# Concatenate
concat = concatenate([dense_name, dense_summary, dense_description, dense_amenities, input_nontext])
# dense_full1 = Dense(256, activation = "relu")(concat)
dense_full2 = Dense(128, activation="relu")(concat) # (dense_full1)
dense_full3 = Dense(64, activation="relu")(dense_full2)
output_layer = Dense(1, activation = "linear")(dense_full3)

model = Model(inputs=[input_name, input_summary, input_description, input_amenities, input_nontext], outputs = output_layer)
model.summary()

In [223]:
start = time.time()

opt = keras.optimizers.Adam(learning_rate=0.01) 

model.compile(optimizer = opt, loss='mean_squared_error', metrics=['mse', 'mae'])

batch_size = 128
epochs = 25

callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=0.01, restore_best_weights=True)]

story = model.fit([padded_X_train_name, padded_X_train_summary, padded_X_train_description, padded_X_train_amenities, X_train], y_train,
                    batch_size = batch_size,
                    epochs = epochs,
                    verbose = 1,
                    validation_split = 0.25,
                  callbacks = callbacks) 

end = time.time()

print(end - start)

Epoch 1/25
260/260 [==============================] - 2955s 11s/step - loss: 3539.8206 - mse: 3539.8206 - mae: 36.9064 - val_loss: 2757.3503 - val_mse: 2757.3503 - val_mae: 31.3115
Epoch 2/25
260/260 [==============================] - 3015s 12s/step - loss: 2654.9353 - mse: 2654.9353 - mae: 31.6175 - val_loss: 2637.4202 - val_mse: 2637.4202 - val_mae: 30.5113
Epoch 3/25
260/260 [==============================] - 3143s 12s/step - loss: 2329.1807 - mse: 2329.1807 - mae: 29.7039 - val_loss: 2637.0850 - val_mse: 2637.0850 - val_mae: 32.2671
Epoch 4/25
260/260 [==============================] - 3083s 12s/step - loss: 2034.5288 - mse: 2034.5288 - mae: 27.5840 - val_loss: 2694.5303 - val_mse: 2694.5303 - val_mae: 32.3450
Epoch 5/25
260/260 [==============================] - 3401s 13s/step - loss: 1822.6223 - mse: 1822.6223 - mae: 26.1869 - val_loss: 2844.7207 - val_mse: 2844.7207 - val_mae: 31.6470
Epoch 6/25
260/260 [==============================] - 3243s 12s/step - loss: 1690.0753 - mse: 1

In [225]:
lstm_val_prediction = model.predict([padded_X_test_name, padded_X_test_summary, padded_X_test_description, padded_X_test_amenities, X_test])
listing_id = pd.DataFrame(X_test_listing_id, columns = ["listing_id"])
lstm_val_prediction = pd.DataFrame(lstm_val_prediction, columns = ["price"])
lstm_val_prediction.describe()

346/346 [==============================] - 270s 781ms/step


,price
count,11057.000000
mean,108.701210
std,70.819878
min,20.272205
25%,54.626667
50%,88.858482
75%,138.898926
max,623.955505
